<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #10: Twitter Jhonathan Giraldo
`Fecha de entrega: Noviembre 14, 2020 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

(Si exponen sus claves, les haré maldades 😈)

❌🎃❌ ¡No Halloween!

In [34]:
import pandas as pd 
import tweepy

In [147]:
tweets = tweepy.Cursor(api.search,
                       q=['WOM'],
                       lang='es',
                       geocode=f'4.60971,-74.08175,45km',
                       tweet_mode='extended',
                       include_rts=False).items(1000)

In [148]:
t = []

for tweet in tweets:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append("Bogotá")
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/status/{tweet.id}")
    
    t.append(temp)

In [149]:
data = pd.DataFrame(t)
data.columns = ['tweet', 'date', 'city', 'user_location', 'id', 'link']
data.head()

,tweet,date,city,user_location,id,link
0,@cardami114 @Fergicienta Pues no le veo lo mal...,2020-11-10 16:01:52,Bogotá,Bogota,1326193364367265795,https://twitter.com/i/web/status/1326193364367...
1,Le contamos los detalles del lanzamiento de WO...,2020-11-10 15:30:44,Bogotá,"Bogotá, Colombia",1326185530594385920,https://twitter.com/i/web/status/1326185530594...
2,@AurelioSuarez @ClaroColombia Tranquilo abueli...,2020-11-10 13:28:41,Bogotá,,1326154817731964930,https://twitter.com/i/web/status/1326154817731...
3,‘Estamos listos para darlo todo por nuestros c...,2020-11-10 12:10:57,Bogotá,Bogota,1326135253107437568,https://twitter.com/i/web/status/1326135253107...
4,Antes de especular sobre Avantel y WOM lean es...,2020-11-10 05:51:19,Bogotá,bogotá,1326039714953093122,https://twitter.com/i/web/status/1326039714953...


In [150]:
data = data[['tweet', 'date', 'city']]
data.head()

,tweet,date,city
0,@cardami114 @Fergicienta Pues no le veo lo mal...,2020-11-10 16:01:52,Bogotá
1,Le contamos los detalles del lanzamiento de WO...,2020-11-10 15:30:44,Bogotá
2,@AurelioSuarez @ClaroColombia Tranquilo abueli...,2020-11-10 13:28:41,Bogotá
3,‘Estamos listos para darlo todo por nuestros c...,2020-11-10 12:10:57,Bogotá
4,Antes de especular sobre Avantel y WOM lean es...,2020-11-10 05:51:19,Bogotá


In [151]:
import re

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweet']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
             
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")
        
    return text, links, hashtags, mentions

In [152]:
data[['clean', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')

data.head()

,tweet,date,city,clean,links,hashtags,mentions
0,@cardami114 @Fergicienta Pues no le veo lo mal...,2020-11-10 16:01:52,Bogotá,Pues no le veo lo malo en caso que fuera Chi...,[],[],"[@cardami114, @Fergicienta]"
1,Le contamos los detalles del lanzamiento de WO...,2020-11-10 15:30:44,Bogotá,Le contamos los detalles del lanzamiento de WO...,[https://t.co/4u4IZjdeCL],[],[]
2,@AurelioSuarez @ClaroColombia Tranquilo abueli...,2020-11-10 13:28:41,Bogotá,Tranquilo abuelito.\n\nYa llegó WOM.\n\nTrae...,[],[],"[@AurelioSuarez, @ClaroColombia]"
3,‘Estamos listos para darlo todo por nuestros c...,2020-11-10 12:10:57,Bogotá,‘Estamos listos para darlo todo por nuestros c...,[https://t.co/YoOWFan6fh],"[#marketingencolombia, #marketing, #marketinge...",[]
4,Antes de especular sobre Avantel y WOM lean es...,2020-11-10 05:51:19,Bogotá,Antes de especular sobre Avantel y WOM lean es...,[https://t.co/UVhxQFTfVm],[],[]


In [153]:
import regex
import emoji

def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

data['emojis'] = data['clean'].apply(lambda text: get_emojis(text))

In [154]:
data.head()

,tweet,date,city,clean,links,hashtags,mentions,emojis
0,@cardami114 @Fergicienta Pues no le veo lo mal...,2020-11-10 16:01:52,Bogotá,Pues no le veo lo malo en caso que fuera Chi...,[],[],"[@cardami114, @Fergicienta]",[]
1,Le contamos los detalles del lanzamiento de WO...,2020-11-10 15:30:44,Bogotá,Le contamos los detalles del lanzamiento de WO...,[https://t.co/4u4IZjdeCL],[],[],[]
2,@AurelioSuarez @ClaroColombia Tranquilo abueli...,2020-11-10 13:28:41,Bogotá,Tranquilo abuelito.\n\nYa llegó WOM.\n\nTrae...,[],[],"[@AurelioSuarez, @ClaroColombia]",[]
3,‘Estamos listos para darlo todo por nuestros c...,2020-11-10 12:10:57,Bogotá,‘Estamos listos para darlo todo por nuestros c...,[https://t.co/YoOWFan6fh],"[#marketingencolombia, #marketing, #marketinge...",[],[]
4,Antes de especular sobre Avantel y WOM lean es...,2020-11-10 05:51:19,Bogotá,Antes de especular sobre Avantel y WOM lean es...,[https://t.co/UVhxQFTfVm],[],[],[]


In [155]:
data.to_excel("tweets_wom.xlsx")

###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

❌🎃❌ ¡No Halloween!

In [156]:
data['fecha'] = data.date.dt.date
temp = pd.DataFrame(data.fecha.value_counts()).reset_index()
temp.columns = ['fecha', 'cnt']
temp = temp.sort_values('fecha')
temp

,fecha,cnt
4,2020-11-03,20
5,2020-11-04,18
6,2020-11-05,11
0,2020-11-06,173
1,2020-11-07,77
3,2020-11-08,20
2,2020-11-09,26
7,2020-11-10,9


In [158]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [159]:
trace = go.Scatter(x=temp.fecha.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.fecha.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Días',
                   line={'color':'cyan'})

layout = go.Layout(title='Número de tuits por día')

fig = go.Figure(data=trace, layout=layout)
iplot(fig)

In [161]:
from collections import Counter

temp_emojis = pd.DataFrame(list(zip(Counter(sum(data.emojis.values,[])).keys(),Counter(sum(data.emojis.values,[])).values())))
temp_emojis.columns = ['emoji', 'cnt']
temp_emojis.sort_values('cnt', ascending=False, inplace=True)
temp_emojis.head()

,emoji,cnt
0,😂,23
1,👉,17
2,🤔,4
3,📢,4
5,🤘🏻,3
